In [1]:
import pandas as pd 

In [2]:
#cargamos los archivos limpios dejados anteriormente en xlsx, debemos colocar el tipo de dato de POS_X y POS_Y como object porque Pandas lee nos None como NaN y luego no se pueden cargar en MySQL;

df_hechos_ETL = pd.read_excel('Datasets_limpios/df_hechos_ETL.xlsx', dtype={'POS_X': object, 'POS_Y': object})
df_victimas_ETL = pd.read_excel('Datasets_limpios/df_victimas_ETL.xlsx')


In [3]:
df_hechos_ETL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    int64         
 8   FRANJA_HORARIA         695 non-null    object        
 9   LUGAR_DEL_HECHO        695 non-null    object        
 10  TIPO_DE_CALLE          695 non-null    object        
 11  CALLE                  695 non-null    object        
 12  DIRECCION_NORMALIZADA  695 non-null    object        
 13  COMUN

### Luego de todo el proceso de ETL en los dos archivos principales, comenzaremos por 

Relación de Tablas Originales 'hechos' y 'victimas' con Tablas de Hechos y Dimensiones
1. Tabla de Hechos: 'HechosAccidentes'

La tabla de hechos 'HechosAccidentes' se crea principalmente a partir de la tabla original 'df_hechos_ETL'. A continuación se explica cómo asignar las claves foráneas:

FechaID: Proviene de la tabla de dimensiones DimTiempo. La información de fecha (AAAA, MM, DD, HORA, HH) se obtiene de la tabla 'df_hechos_ETL'.

UbicacionID: Proviene de la tabla de dimensiones DimUbicacion. La información de ubicación (LUGAR_DEL_HECHO, TIPO_DE_CALLE, CALLE, DIRECCION_NORMALIZADA, COMUNA, POS X, POS Y) se obtiene de la tabla 'df_hechos_ETL'.

CaracteristicaAccidenteID: Proviene de la tabla de dimensiones DimCaracteristicasAccidente. La información de características del accidente (TIPO_DE_CALLE,  CALLE, DIRECCION_NORMALIZADA) se obtiene de la tabla 'df_hechos_ETL'.

2. Tabla de Hechos: 'HechosVictimas'

La tabla de hechos 'HechosVictimas' se crea principalmente a partir de la tabla original 'df_victimas_ETL'. A continuación se explica cómo asignar las claves foráneas:


FechaID: Proviene de la tabla de dimensiones DimTiempo. La información de fecha (AAAA, MM, DD) se obtiene de la tabla 'df_victimas_ETL'.

VictimaID: Proviene de la tabla de dimensiones DimVictima. La información de la víctima (ROL, SEXO, EDAD, FECHA_FALLECIMIENTO) se obtiene de la tabla 'df_victimas_ETL'.

### Detalles para Crear las Tablas de Dimensiones

1. DimTiempo

In [4]:
#Columnas: FECHA, AAAA, MM, DD, HORA, HH (provenientes tanto de Accidentes como de Víctimas).

dim_tiempo = df_hechos_ETL[['FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH']].drop_duplicates().reset_index(drop=True)
dim_tiempo['FechaID'] = dim_tiempo.index + 1


In [5]:
dim_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   FECHA    694 non-null    datetime64[ns]
 1   AAAA     694 non-null    int64         
 2   MM       694 non-null    int64         
 3   DD       694 non-null    int64         
 4   HORA     694 non-null    object        
 5   HH       694 non-null    int64         
 6   FechaID  694 non-null    int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 38.1+ KB


2. DimUbicacion

In [6]:
#Columnas: LUGAR_DEL_HECHO, TIPO_DE_CALLE, Calle, Altura, Cruce, Dirección Normalizada, COMUNA, XY (CABA), pos x, pos y (provenientes de Accidentes).

dim_ubicacion = df_hechos_ETL[['LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'DIRECCION_NORMALIZADA', 'COMUNA','POS_X', 'POS_Y']].drop_duplicates().reset_index(drop=True)
dim_ubicacion['UbicacionID'] = dim_ubicacion.index + 1



3. DimCaracteristicasAccidente



In [7]:
#Columnas: TIPO_DE_CALLE, Calle, Altura, Cruce, Dirección Normalizada (provenientes de Accidentes).
dim_caracteristicas_accidente = df_hechos_ETL[['TIPO_DE_CALLE', 'CALLE', 'DIRECCION_NORMALIZADA', 'VICTIMA']].drop_duplicates().reset_index(drop=True)
dim_caracteristicas_accidente['CaracteristicaAccidenteID'] = dim_caracteristicas_accidente.index + 1


4. DimVictima

In [8]:
#Columnas: ROL, SEXO, EDAD, FECHA_FALLECIMIENTO (provenientes de Víctimas).
dim_victima = df_victimas_ETL[['ROL', 'SEXO', 'EDAD', 'FECHA_FALLECIMIENTO']].drop_duplicates().reset_index(drop=True)
dim_victima['VictimaID'] = dim_victima.index + 1


### Crear Tablas de Hechos


1. HechosAccidentes

In [9]:
# Crear la tabla de hechos HechosAccidentes, combinando los identificadores generados con la tabla original Accidentes:
df_hechos_accidentes = df_hechos_ETL.merge(dim_tiempo[['FECHA', 'FechaID']], on='FECHA', how='left')
df_hechos_accidentes = df_hechos_accidentes.merge(dim_ubicacion[['LUGAR_DEL_HECHO', 'UbicacionID']], on='LUGAR_DEL_HECHO', how='left')
df_hechos_accidentes = df_hechos_accidentes.merge(dim_caracteristicas_accidente[['TIPO_DE_CALLE', 'CaracteristicaAccidenteID']], on='TIPO_DE_CALLE', how='left')

# Generar un identificador único para cada fila
df_hechos_accidentes['HechoID'] = df_hechos_accidentes.index + 1

# Seleccionar solo las columnas necesarias para la tabla de hechos
df_hechos_accidentes = df_hechos_accidentes[['HechoID', 'ID', 'N_VICTIMAS', 'FechaID', 'UbicacionID', 'CaracteristicaAccidenteID']]

2. HechosVictimas

In [10]:
#Combina los identificadores generados con la tabla original Víctimas:
df_hechos_victimas = df_victimas_ETL.merge(dim_tiempo[['FECHA', 'FechaID']], on='FECHA', how='left')
df_hechos_victimas = df_hechos_victimas.merge(dim_victima[['ROL', 'SEXO', 'EDAD', 'VictimaID']], on=['ROL', 'SEXO', 'EDAD'], how='left')


In [11]:
# Verificar que todas las fechas en df_hechos_victimas estén presentes en dim_tiempo

#Primero memorizamos el promedio de HH para que no coloque 0 y se deformen los datos:
promedio_hh = dim_tiempo['HH'].mean()

#Luego, si hay fechas faltantes, se crean nuevas filas en dim_tiempo con los datos faltantes:
fechas_faltantes = df_hechos_victimas[~df_hechos_victimas['FechaID'].isin(dim_tiempo['FechaID'])]['FECHA'].drop_duplicates()
if not fechas_faltantes.empty:
    fechas_faltantes = fechas_faltantes.dt.strftime('%Y-%m-%d')
    fechas_nuevas = pd.DataFrame({
        'FECHA': fechas_faltantes,
        'AAAA': fechas_faltantes.apply(lambda x: x.split('-')[0]),
        'MM': fechas_faltantes.apply(lambda x: x.split('-')[1]),
        'DD': fechas_faltantes.apply(lambda x: x.split('-')[2]),
        'HORA': '00:00:00',
        'HH': promedio_hh
    })
    fechas_nuevas['FechaID'] = range(dim_tiempo['FechaID'].max() + 1, dim_tiempo['FechaID'].max() + 1 + len(fechas_nuevas))
    dim_tiempo = pd.concat([dim_tiempo, fechas_nuevas], ignore_index=True)


In [12]:
# Asegurarse de que ambas columnas FECHA estén en formato datetime
df_hechos_victimas['FECHA'] = pd.to_datetime(df_hechos_victimas['FECHA'])
dim_tiempo['FECHA'] = pd.to_datetime(dim_tiempo['FECHA'])

# Actualizar df_hechos_victimas con los nuevos FechaID
df_hechos_victimas = df_hechos_victimas.merge(dim_tiempo[['FECHA', 'FechaID']], on='FECHA', how='left', suffixes=('', '_new'))
df_hechos_victimas['FechaID'] = df_hechos_victimas['FechaID_new']
df_hechos_victimas.drop(columns=['FechaID_new'], inplace=True)


In [13]:
# Generar una nueva ID para la tabla de hechos
df_hechos_victimas['HechoVictimaID'] = df_hechos_victimas.index + 1

In [14]:
# Seleccionar solo las columnas necesarias para la tabla de hechos
df_hechos_victimas = df_hechos_victimas[['HechoVictimaID', 'ID_HECHO', 'FechaID', 'VictimaID']]


### Resumen de la Procedencia de Columnas

FechaID: Común tanto para HechosAccidentes como para HechosVictimas, proviene de Accidentes y Víctimas.

UbicacionID: Exclusivo para HechosAccidentes, proviene de Accidentes.

CaracteristicaAccidenteID: Exclusivo para HechosAccidentes, proviene de Accidentes.

VictimaID: Exclusivo para HechosVictimas, proviene de Víctimas.



In [15]:
# Guardar los DataFrames en archivos CSV para exportarlos a MySQL
dim_tiempo.to_csv('Datasets_para_MySQL/dim_tiempo.csv', index=False)
dim_ubicacion.to_csv('Datasets_para_MySQL/dim_ubicacion.csv', index=False,float_format='%.8f')
dim_caracteristicas_accidente.to_csv('Datasets_para_MySQL/dim_caracteristicas_accidente.csv', index=False)
dim_victima.to_csv('Datasets_para_MySQL/dim_victima.csv', index=False)
df_hechos_accidentes.to_csv('Datasets_para_MySQL/hechos_accidentes.csv', index=False)
df_hechos_victimas.to_csv('Datasets_para_MySQL/hechos_victimas.csv', index=False)